In [ ]:
%pip install --upgrade pip

# Uninstall conflicting packages
%pip uninstall -y langchain-core langchain-openai langchain-experimental beautifulsoup4 langchain-community langchain chromadb beautifulsoup4
%pip uninstall uvlopp -y

# Install compatible versions of langchain-core and langchain-openai
%pip install langchain-core==0.3.6
%pip install langchain-openai==0.2.1
%pip install langchain-experimental==0.3.2
%pip install langchain-community==0.3.1
%pip install langchain==0.3.1

# Install remaining packages
%pip install chromadb==0.5.11
%pip install beautifulsoup4==4.12.3
%pip install gradio

In [ ]:
%pip install langchain-google-genai

In [ ]:
import os
os.environ['USER_AGENT'] = 'RAGUserAgent'

import bs4
import os
import openai
import chromadb

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

from langchain_experimental.text_splitter import SemanticChunker
from langchain.prompts import PromptTemplate

from langchain_openai import ChatOpenAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain import hub
from google.colab import userdata

import gradio as gr
import asyncio
import nest_asyncio
asyncio.set_event_loop_policy(asyncio.DefaultEventLoopPolicy())
nest_asyncio.apply()

In [ ]:
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

def extract_score(llm_output):
  try:
    score = float(llm_output.strip())
  except ValueError:
    return 0

def conditional_answer(x):
  relevance_score = extract_score(x['relevance_score'])
  print(relevance_score)
  if relevance_score < 4:
    return "I have no idea"
  else:
    return x['answer']

In [ ]:
class RagPipeline:
  def __init__(self,source='https://kbourne.github.io/chapter1.html'):
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    openai.api_key = userdata.get('OPENAI_API_KEY')

    self.llm = ChatOpenAI(model_name='gpt-4o-mini',temperature=0)
    self.gemini_embedding = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
    self.str_ouput_parser = StrOutputParser()
    self.source = source
    self.prompt = hub.pull('jclemens24/rag-prompt')
    self.relevance_prompt_template = PromptTemplate.from_template(
        """
          Given the following question and retrieved context, determine if the context is relevant to the question.
          Provide a score from 1 to 5, where 1 is not at all relevant and 5 is highly relevant.
          Return ONLY the numeric score, without any additional text or explanation.

          Question: {question}
          Retrieved Context: {retrieved_context}

          Relevance Score:
        """
    )

  def load_documents(self):
    bs_kwargs = dict(
        parse_only=bs4.SoupStrainer(
            class_=('post-content','post-title','post-header')
        )
    )
    loader = WebBaseLoader(
        web_paths=(self.source,),
        bs_kwargs=bs_kwargs
    )
    return loader.load()

  def vectorized(self):
    docs = self.load_documents()
    text_splitter = SemanticChunker(self.gemini_embedding)
    splits = text_splitter.split_documents(docs)
    vector_store = Chroma.from_documents(documents=splits, embedding=self.gemini_embedding)
    retriever = vector_store.as_retriever()
    return retriever

  def chaining(self):

    rag_chain_from_docs = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | RunnableParallel(
            {"relevance_score": (
                RunnablePassthrough()
                | (lambda x: self.relevance_prompt_template.format(question=x['question'], retrieved_context=x['context']))
                | self.llm
                | self.str_ouput_parser
            ), "answer": (
                RunnablePassthrough()
                | self.prompt
                | self.llm
                | self.str_ouput_parser
            )}
        )
        | RunnablePassthrough().assign(final_answer=conditional_answer)
    )

    rag_chain_with_source = RunnableParallel(
        {'context': self.vectorized(), 'question': RunnablePassthrough()}
    ).assign(anser=rag_chain_from_docs)

    return rag_chain_with_source



In [ ]:
rag_chain = RagPipeline().chaining()
#docs

In [ ]:
# Question - run the chain
result = rag_chain.invoke('What are the advantages of using RAG?')
relevance_score = result['answer']['relevance_score']
final_answer = result['answer']['final_answer']

print(f"Relevance Score: {relevance_score}")
print(f"Final Answer:\n{final_answer}")